In [241]:
import sys
sys.version_info

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)

In [284]:
import numpy as np
import pandas as pd 

In [286]:
pd.set_option('display.max_columns', None)

In [243]:
#read in the data
marathon_data = pd.read_csv("data.csv")
marathon_data = marathon_data.dropna()
marathon_data.head()

,Id,Name,Age Category,Sex,Rank,Time,Pace,Year
0,1,carlos enrique portugal quintero,30,M,1263,04:15:00,09:44,2016
1,2,leonel furlan,60,M,2289,05:05:42,11:41,2016
2,3,meredith gray,30,F,2276,05:04:34,11:38,2016
3,4,jason ainspan,35,M,670,3:52:57,08:53,2015
4,4,jason ainspan,35,M,2390,05:13:30,11:58,2016


In [244]:
marathon_data['Time'] = pd.to_timedelta(marathon_data['Time'])
#marathon_data['Pace'] = pd.to_timedelta(marathon_data['Pace'])
marathon_data['Time'].head()
#marathon_data['Pace'].head()

0   04:15:00
1   05:05:42
2   05:04:34
3   03:52:57
4   05:13:30
Name: Time, dtype: timedelta64[ns]

In [245]:
marathon_data.groupby('Year').get_group(2016)['Time'].mean()

Timedelta('0 days 04:32:55.103116')

In [246]:
marathon_data.groupby('Year').get_group(2015)['Time'].mean()

Timedelta('0 days 04:30:48.480377')

In [247]:
for yr in marathon_data.groupby('Year').groups: 
    print(yr, str(marathon_data.groupby('Year').get_group(yr)['Time'].mean()))
#here, we are trying to figure out which year was the half marathon (2013)
#Let's drop that year
marathon_data = marathon_data[marathon_data.Year != 2013]
#marathon_data.Year.value_counts()

2016 0 days 04:32:55.103116
2003 0 days 04:18:00.990378
2004 0 days 04:28:34.934463
2005 0 days 04:25:55.415105
2006 0 days 04:37:25.943564
2007 0 days 04:43:34.228222
2008 0 days 04:28:04.394986
2009 0 days 04:29:01.636166
2010 0 days 04:36:43.382291
2011 0 days 04:27:12.712468
2012 0 days 04:35:52.052658
2013 0 days 01:51:27.648069
2014 0 days 04:43:50.870375
2015 0 days 04:30:48.480377


In [248]:
marathon_data.Name.value_counts().sort_values().tail()
#Note: we see that some of the participants have the name set to private. we should probably drop these 
marathon_data = marathon_data[marathon_data.Name != "private"]
#marathon_data.Name.value_counts().sort_values().tail()
marathon_data = marathon_data[marathon_data.Name.str.contains("#") == False]
#get rid of one garbage name

In [249]:
marathon_data.sort_values("Name").head()

,Id,Name,Age Category,Sex,Rank,Time,Pace,Year
3175,1941,aaron allen,25,M,267,03:28:52,07:59,2016
735,449,aaron anderson,35,M,2286,05:05:22,11:40,2016
16136,10644,aaron benscher,32,M,2048,05:30:48,12:37,2007
26891,19272,aaron butler,33,M,2109,04:35:30,10:30,2011
26892,19272,aaron butler,34,M,2813,05:03:01,11:33,2012


In [250]:
marathon_data.head()
marathon_data.tail()

,Id,Name,Age Category,Sex,Rank,Time,Pace,Year
38799,30413,mike willyoung,25,M,1455,04:27:21,10:12,2015
38800,30414,cheyenne smith,30,M,628,03:51:14,08:49,2015
38801,30415,regine wolf,45,F,511,03:44:54,08:35,2015
38802,30416,dimitri boutsikakis,35,M,640,03:51:36,08:50,2015
38803,30417,rodrigo pincheira,40,M,10,02:50:26,06:30,2015


In [251]:
marathon_data.sort_values('Time').head()
#This looks good (all the very low times are winners of the race, or close to it)

,Id,Name,Age Category,Sex,Rank,Time,Pace,Year
5268,3243,david ruto,23,M,1,02:12:22,05:03,2003
7631,4390,otavio dos santos p,29,M,2,02:12:23,05:03,2003
6621,3816,kiplimo rono,32,M,3,02:12:40,05:03,2003
24807,17531,tesfaye alemayehu,25,M,1,02:12:57,05:04,2011
8341,4853,william gomez amori,25,M,1,02:14:42,05:08,2004


In [252]:
marathon_data.groupby('Name').size().sort_values().tail(15)
#This is reasonable, we se that some people participated in the marathon each year that it occured

Name
wayne wright       12
joseph scott       12
mark platt         12
mike culp          12
juan carson        12
dennis marsella    12
matt duncan        12
bouchra franz      12
kenneth royster    12
tammy lopez        13
alan bloom         13
alexis garcia      13
tony jones         13
paul schoenlaub    13
norberto milan     13
dtype: int64

In [253]:
num_marathons = marathon_data.groupby('Name').size().reset_index()
num_marathons.columns = ["Name","Num"]
num_marathons.head()

,Name,Num
0,aaron allen,1
1,aaron anderson,1
2,aaron benscher,1
3,aaron butler,2
4,aaron carter,1


In [255]:
marathon_data.drop_index = marathon_data.Name

In [262]:
del marathon_data["Pace"]
marathon_data.head()

KeyError: 'Pace'

In [273]:
marathon_data.head()

,Id,Name,Age Category,Sex,Rank,Time,Year
0,1,carlos enrique portugal quintero,30,M,1263,04:15:00,2016
1,2,leonel furlan,60,M,2289,05:05:42,2016
2,3,meredith gray,30,F,2276,05:04:34,2016
3,4,jason ainspan,35,M,670,03:52:57,2015
4,4,jason ainspan,35,M,2390,05:13:30,2016


In [356]:
data_pivot = pd.pivot_table(marathon_data, index = ["Name", "Sex"], columns = "Year", values = ["Rank","Age Category", "Time"], aggfunc= "first")

Rank                                                  \
Year                2003  2004  2005  2006  2007  2008  2009  2010  2011   
Name           Sex                                                         
aaron allen    M    None  None  None  None  None  None  None  None  None   
aaron anderson M    None  None  None  None  None  None  None  None  None   
aaron benscher M    None  None  None  None  2048  None  None  None  None   
aaron butler   M    None  None  None  None  None  None  None  None  2109   
aaron carter   M    None   803  None  None  None  None  None  None  None   

                                           Age Category                    \
Year                2012  2014  2015  2016         2003  2004  2005  2006   
Name           Sex                                                          
aaron allen    M    None  None  None   267         None  None  None  None   
aaron anderson M    None  None  None  2286         None  None  None  None   
aaron benscher M    None  None  None  None         None  None  None  None   
aaron butler   M    2813  None  None  None         None  None  None  None   
aaron carter   M    None  None  None  None         None    19  None  None   

                                                                          \
Year                2007  2008  2009  2010  2011  2012  2014  2015  2016   
Name           Sex                                                         
aaron allen    M    None  None  None  None  None  None  None  None    25   
aaron anderson M    None  None  None  None  None  None  None  None    35   
aaron benscher M      32  None  None  None  None  None  None  None  None   
aaron butler   M    None  None  None  None    33    34  None  None  None   
aaron carter   M    None  None  None  None  None  None  None  None  None   

                    Time                                                      \
Year                2003             2004  2005  2006             2007  2008   
Name           Sex                                                             
aaron allen    M    None             None  None  None             None  None   
aaron anderson M    None             None  None  None             None  None   
aaron benscher M    None             None  None  None  0 days 05:30:48  None   
aaron butler   M    None             None  None  None             None  None   
aaron carter   M    None  0 days 04:17:16  None  None             None  None   

                                                                              \
Year                2009  2010             2011             2012  2014  2015   
Name           Sex                                                             
aaron allen    M    None  None             None             None  None  None   
aaron anderson M    None  None             None             None  None  None   
aaron benscher M    None  None             None             None  None  None   
aaron butler   M    None  None  0 days 04:35:30  0 days 05:03:01  None  None   
aaron carter   M    None  None             None             None  None  None   

                                     
Year                           2016  
Name           Sex                   
aaron allen    M    0 days 03:28:52  
aaron anderson M    0 days 05:05:22  
aaron benscher M               None  
aaron butler   M               None  
aaron carter   M               None

In [357]:
data_pivot["Num Marathons"] = data_pivot.apply(lambda x: (x.count() - 1) / 2, axis = 1)

Rank                                                  \
Year                2003  2004  2005  2006  2007  2008  2009  2010  2011   
Name           Sex                                                         
aaron allen    M    None  None  None  None  None  None  None  None  None   
aaron anderson M    None  None  None  None  None  None  None  None  None   
aaron benscher M    None  None  None  None  2048  None  None  None  None   
aaron butler   M    None  None  None  None  None  None  None  None  2109   
aaron carter   M    None   803  None  None  None  None  None  None  None   

                                           Age Category                    \
Year                2012  2014  2015  2016         2003  2004  2005  2006   
Name           Sex                                                          
aaron allen    M    None  None  None   267         None  None  None  None   
aaron anderson M    None  None  None  2286         None  None  None  None   
aaron benscher M    None  None  None  None         None  None  None  None   
aaron butler   M    2813  None  None  None         None  None  None  None   
aaron carter   M    None  None  None  None         None    19  None  None   

                                                                          \
Year                2007  2008  2009  2010  2011  2012  2014  2015  2016   
Name           Sex                                                         
aaron allen    M    None  None  None  None  None  None  None  None    25   
aaron anderson M    None  None  None  None  None  None  None  None    35   
aaron benscher M      32  None  None  None  None  None  None  None  None   
aaron butler   M    None  None  None  None    33    34  None  None  None   
aaron carter   M    None  None  None  None  None  None  None  None  None   

                    Time                                                      \
Year                2003             2004  2005  2006             2007  2008   
Name           Sex                                                             
aaron allen    M    None             None  None  None             None  None   
aaron anderson M    None             None  None  None             None  None   
aaron benscher M    None             None  None  None  0 days 05:30:48  None   
aaron butler   M    None             None  None  None             None  None   
aaron carter   M    None  0 days 04:17:16  None  None             None  None   

                                                                              \
Year                2009  2010             2011             2012  2014  2015   
Name           Sex                                                             
aaron allen    M    None  None             None             None  None  None   
aaron anderson M    None  None             None             None  None  None   
aaron benscher M    None  None             None             None  None  None   
aaron butler   M    None  None  0 days 04:35:30  0 days 05:03:01  None  None   
aaron carter   M    None  None             None             None  None  None   

                                    Num Marathons  
Year                           2016                
Name           Sex                                 
aaron allen    M    0 days 03:28:52           1.0  
aaron anderson M    0 days 05:05:22           1.0  
aaron benscher M               None           1.0  
aaron butler   M               None           2.5  
aaron carter   M               None           1.0

In [362]:
def lastfilled(row):
    key = 2016
    while row.loc[key] == "None": 
        key = key-1
        if (key == 2013):
            key = key - 1
    return (2017 - key)
        
def timelast(row):
    key = 2016
    while row.loc[key] == "None": 
        key = key-1
        if (key == 2013):
            key = key - 1
    return row.loc[key]

data_pivot["Years Since Last"] = data_pivot.apply(lambda x: lastfilled(x["Rank"]), axis = 1)
data_pivot["Time of Last"] = data_pivot.apply(lambda x: lastfilled(x["Rank"]), axis = 1)
#Add in a column for years since last completed marathon
#And another one for time of last marathon

In [363]:
data_pivot.head()

Rank                                                  \
Year                2003  2004  2005  2006  2007  2008  2009  2010  2011   
Name           Sex                                                         
aaron allen    M    None  None  None  None  None  None  None  None  None   
aaron anderson M    None  None  None  None  None  None  None  None  None   
aaron benscher M    None  None  None  None  2048  None  None  None  None   
aaron butler   M    None  None  None  None  None  None  None  None  2109   
aaron carter   M    None   803  None  None  None  None  None  None  None   

                                           Age Category                    \
Year                2012  2014  2015  2016         2003  2004  2005  2006   
Name           Sex                                                          
aaron allen    M    None  None  None   267         None  None  None  None   
aaron anderson M    None  None  None  2286         None  None  None  None   
aaron benscher M    None  None  None  None         None  None  None  None   
aaron butler   M    2813  None  None  None         None  None  None  None   
aaron carter   M    None  None  None  None         None    19  None  None   

                                                                          \
Year                2007  2008  2009  2010  2011  2012  2014  2015  2016   
Name           Sex                                                         
aaron allen    M    None  None  None  None  None  None  None  None    25   
aaron anderson M    None  None  None  None  None  None  None  None    35   
aaron benscher M      32  None  None  None  None  None  None  None  None   
aaron butler   M    None  None  None  None    33    34  None  None  None   
aaron carter   M    None  None  None  None  None  None  None  None  None   

                    Time                                                      \
Year                2003             2004  2005  2006             2007  2008   
Name           Sex                                                             
aaron allen    M    None             None  None  None             None  None   
aaron anderson M    None             None  None  None             None  None   
aaron benscher M    None             None  None  None  0 days 05:30:48  None   
aaron butler   M    None             None  None  None             None  None   
aaron carter   M    None  0 days 04:17:16  None  None             None  None   

                                                                              \
Year                2009  2010             2011             2012  2014  2015   
Name           Sex                                                             
aaron allen    M    None  None             None             None  None  None   
aaron anderson M    None  None             None             None  None  None   
aaron benscher M    None  None             None             None  None  None   
aaron butler   M    None  None  0 days 04:35:30  0 days 05:03:01  None  None   
aaron carter   M    None  None             None             None  None  None   

                                    Num Marathons Years Since Last  \
Year                           2016                                  
Name           Sex                                                   
aaron allen    M    0 days 03:28:52           1.0                1   
aaron anderson M    0 days 05:05:22           1.0                1   
aaron benscher M               None           1.0                1   
aaron butler   M               None           2.5                1   
aaron carter   M               None           1.0                1   

                   Time of Last  
Year                             
Name           Sex               
aaron allen    M              1  
aaron anderson M              1  
aaron benscher M              1  
aaron butler   M              1  
aaron carter   M              1